In [1]:
import numpy as np
import pandas as pd
import math
import itertools
from sklearn.metrics import DistanceMetric
import networkx as nx
from numpy import inf
from scipy import linalg
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 



In [2]:
  # Uploading the files
df = pd.read_csv("HeartFailure.csv")
df_cluster = df.iloc[:,1:13]
df_cluster.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8


In [3]:
# coverting the input csv file into a graph
'''  '''
def getGraph(df,measure) :
    
    dist = DistanceMetric.get_metric(measure)
    matrix_dist = dist.pairwise(df.to_numpy())
    Graph=nx.from_numpy_matrix(matrix_dist)
    #draw_graph(Graph)
    
    return matrix_dist, Graph
    

In [4]:
#Function to draw graph
def draw_graph(G):
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

In [5]:
matrix_dist, Graph = getGraph(df_cluster,"euclidean")
matrix_dist

array([[     0.        ,   7461.94827916, 103000.92333741, ...,
        477002.37119768, 125013.7195001 , 130000.88170889],
       [  7461.94827916,      0.        , 101651.22697499, ...,
        478677.19981275, 123478.57896239, 131865.22819429],
       [103000.92333741, 101651.22697499,      0.        , ...,
        580003.22319902,  22118.19755337, 233000.17313961],
       ...,
       [477002.37119768, 478677.19981275, 580003.22319902, ...,
             0.        , 602000.10390727, 347005.00688411],
       [125013.7195001 , 123478.57896239,  22118.19755337, ...,
        602000.10390727,      0.        , 255009.63747286],
       [130000.88170889, 131865.22819429, 233000.17313961, ...,
        347005.00688411, 255009.63747286,      0.        ]])

In [6]:
#equation fro adjacency matrix
def equation(point):
    n = 0.1
    function = np.exp(-((point ** 2) / (2. * n ** 2)))
    return function 
    

In [12]:
#Getting adjacency matrix for all the points in csv
def getAjacency(matrix):
    adacency_matrix = []
    for row in matrix_dist:
        x = []
        for point in row:
            x.append(equation(point))
            adacency_matrix.append(x)
    np.array(adacency_matrix)
    
    return adacency_matrix 

In [ ]:
adacency_matrix= getAjacency(matrix_dist)
print(adacency_matrix)


In [ ]:
def getLaplacian(Graph,matrix_dist):
    D = Graph.degree()
    degrees = [val for (node, val) in Graph.degree()]
    D = np.zeros((len(degrees), len(degrees)))
    np.fill_diagonal(D, degrees)
#     print('degree matrix:')
    print(D)
    L = D - matrix_dist
    
#Normalized laplacian matrix
    D_half = linalg.fractional_matrix_power(D, -0.5)
    LN = np.matmul(np.matmul(D_half, L), D_half)
    
#  print('laplacian matrix:')
    print(LN)
    return D,L,LN


In [ ]:
def getEigen(LN):
    e, v = np.linalg.eig(LN)
        # eigenvalues
    print('eigenvalues:')
    print(e)
        # eigenvectors
    print('eigenvectors:')
    print(v)
    return e,v
    

In [ ]:
D,L,LN = getLaplacian(Graph,matrix_dist)

In [ ]:
e,v = getEigen(LN)

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(121)
plt.plot(e)
ax1.title.set_text('eigenvalues')
i = np.where(e < 10e-6)[0]
ax2 = plt.subplot(122)
plt.plot(v[:, i[0]])
fig.tight_layout()
plt.show()

In [ ]:
U = np.array(v[:, i[0]])
km = KMeans(init='k-means++', n_clusters=3)
df['clusters'] = km.fit_predict(U)
df